In [1]:
# installs after TF setup https://www.tensorflow.org/install/pip 

# !pip install pandas
# !pip install mne
# !pip install scikit-learn

In [2]:
import os, glob, platform
import numpy as np
import numpy.matlib
import pickle
import pandas as pd
import pathlib
import matplotlib
import matplotlib.pyplot as plt
import mne
mne.__version__
from mne.viz import plot_alignment, snapshot_brain_montage
import shutil
from mne.datasets import eegbci
from sklearn.model_selection import train_test_split

# from mne_bids import write_raw_bids, BIDSPath, print_dir_tree, make_dataset_description
# from mne_bids.stats import count_events
import sys


In [3]:
path_utils = '/decoding_toolbox_py/helper_funcs' 
sys.path.append(path_utils)

In [4]:
''' VARIABLES '''

dataset = 'eeg'

amount_of_subjects = 4 # Change the range so the process is faster
if amount_of_subjects > 26: amount_of_subjects = 26
subjs_list = ['s{:02d}'.format(i) for i in range(1, amount_of_subjects+1) if i != 6 ] 
print(subjs_list)
nSubj = len(subjs_list)

numC = 8

angles = [i * 180./numC for i in range(numC)]

x_labels = np.array(angles)

resample = True # speeds up the procees but showing worse results overall
if resample: resample_frequency = 20 # in Hz, original freq is 500Hz

cfg_stim = dict()
cfg_stim['kappa'] = 4
cfg_stim['NumC'] = numC
cfg_stim['Tuning'] = 'vonmises'
# cfg_stim['Tuning'] = 'halfRectCos'
cfg_stim['offset'] = 0

cfg_train = dict()
cfg_train['gamma'] = 0.1
cfg_train['demean'] = True
cfg_train['returnPattern'] = True

cfg_test = dict()
cfg_test['demean'] = 'traindata'

['s01', 's02', 's03', 's04']


In [5]:
'''EEG Dataset'''
def read_data(
        number_of_repetition=3,
        resample=False,
        resample_frequency = 20,
        subjs_list = subjs_list,
        task = 'main'
        
        ):
    path = 'Cond_CJ_EEG'

    epochs = []
    all_epochs = []
    all_rawdata = []
    all_st_epochs = []
    all_st_rawdata = []
    for subject_id in subjs_list:
        preproc_path = os.path.join(path, subject_id)

        if task == 'main':
            epoch = mne.read_epochs(os.path.join(preproc_path, 'main_epo.fif'), verbose=False)
            
            if resample: 
                print('Frequency before:', epoch.info['sfreq'])
                epoch = epoch.resample(resample_frequency)
                print('Frequency after:' ,epoch.info['sfreq'])

            epochs.append(epoch.average())
            all_epochs.append(epoch)
            all_rawdata.append({'epoch_dat': epoch.get_data(), 'metadata': epoch.metadata})
            
        if task == 'stim':
        
            st_epoch = mne.read_epochs(os.path.join(preproc_path, 'mainstim_epo.fif'), verbose=False)
            # print(st_epoch.info['sfreq'])
            if resample: 
                print('Frequency before:', st_epoch.info['sfreq'])
                st_epoch = st_epoch.resample(resample_frequency)
                print('Frequency after:' ,st_epoch.info['sfreq'])
                
            all_st_epochs.append(st_epoch)
            all_st_rawdata.append(
                {
                'epoch_dat': st_epoch.get_data()[st_epoch.metadata['nrep'] == number_of_repetition,:,:] ,
                'metadata': st_epoch.metadata[st_epoch.metadata['nrep'] == number_of_repetition]
                }
                )
    if task == 'main':
        return all_rawdata
    else:
        return all_st_rawdata


In [6]:
all_rawdata = read_data (task = 'main', resample=False, resample_frequency=20)

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


In [7]:
all_rawdata[0]['metadata'].columns

Index(['index', 'subj', 'nblock', 'ntrial', 'nrep', 'trial_type', 'cond-1',
       'cond', 'rDV', 'DV', 'resp', 'deci-2', 'deci-1', 'deci', 'corr-1',
       'r_map', 'correct', 'confi', 'RT', 'd1', 'conf_lvl', 'correct-1', 'd2',
       'd3', 'd4', 'd5', 'd6', 'o1', 'o2', 'o3', 'o4', 'o5', 'o6', 'confi-1',
       'conf_lvl-1'],
      dtype='object')

In [8]:
A = np.random.random((3,2,5))
print(A)
A = A.reshape((6,5))
print(A.shape)
A

[[[0.64078111 0.70981593 0.35463163 0.00926    0.46787156]
  [0.48146418 0.4362488  0.32133827 0.93820612 0.53717294]]

 [[0.13604671 0.85857681 0.06331427 0.14165666 0.95519275]
  [0.15166693 0.77787975 0.45766146 0.87659091 0.21268633]]

 [[0.33383336 0.71446525 0.33882879 0.36343169 0.58605082]
  [0.06291592 0.02409186 0.85775097 0.87714349 0.68626269]]]
(6, 5)


array([[0.64078111, 0.70981593, 0.35463163, 0.00926   , 0.46787156],
       [0.48146418, 0.4362488 , 0.32133827, 0.93820612, 0.53717294],
       [0.13604671, 0.85857681, 0.06331427, 0.14165666, 0.95519275],
       [0.15166693, 0.77787975, 0.45766146, 0.87659091, 0.21268633],
       [0.33383336, 0.71446525, 0.33882879, 0.36343169, 0.58605082],
       [0.06291592, 0.02409186, 0.85775097, 0.87714349, 0.68626269]])

In [9]:
b = np.random.random((5))
b = np.tile(b,3)
b

array([0.95233926, 0.03141849, 0.7166512 , 0.87337303, 0.8038557 ,
       0.95233926, 0.03141849, 0.7166512 , 0.87337303, 0.8038557 ,
       0.95233926, 0.03141849, 0.7166512 , 0.87337303, 0.8038557 ])

In [10]:
neural_network = True
if neural_network:
    X = all_rawdata[0]['epoch_dat']
    y = all_rawdata[0]['metadata']['deci']
    for i in range(1, nSubj):
        X = np.concatenate((X, all_rawdata[i]['epoch_dat']), axis=0)
        y = np.concatenate((y, all_rawdata[i]['metadata']['deci']))
    print(X.shape, y.shape)
    numF, numC, numT = X.shape


(1003, 32, 2876) (1003,)


In [11]:
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# MODEL
model = keras.Sequential()
model.add(layers.LSTM(units=128, return_sequences=True, input_shape=(numC, numT)))
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(units=32))
model.add(layers.Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model with callbacks
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# Load the best saved model
best_model = keras.models.load_model('best_model.h5')

# Evaluate the model
loss, accuracy = best_model.evaluate(X_test, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)


2023-06-05 13:14:37.405614: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-05 13:14:37.433303: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-05 13:14:37.865509: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-05 13:14:38.348746: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.

Epoch 1/50


2023-06-05 13:14:39.499057: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-05 13:14:39.499936: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-05 13:14:39.500607: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

InternalError: Graph execution error:

Detected at node 'StatefulPartitionedCall_7' defined at (most recent call last):
    File "/home/jun/miniconda3/envs/thesis-tf-gpu/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/jun/miniconda3/envs/thesis-tf-gpu/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/jun/miniconda3/envs/thesis-tf-gpu/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/jun/miniconda3/envs/thesis-tf-gpu/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/jun/miniconda3/envs/thesis-tf-gpu/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/jun/miniconda3/envs/thesis-tf-gpu/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
     

RF all subjects 80-20 = 0.556809024979855

RF, Logistic, SVC = 1 sub 0.66

XGB 1 sub 0.62

XGB 26 0.5495568090249798

Ada 0.6

GB 0.62

0.66


catboost and RF both on more data with stacked are perfoming the same

next im gonna add labels for the subjects


In [ ]:
# X = all_rawdata[0]['epoch_dat']
# print(X.shape)
# # X = X.reshape(X.shape[0],-1)
# y = all_rawdata[0]['metadata']['deci']
# 
# X2 = all_rawdata[2]['epoch_dat' ]
# y2 = all_rawdata[2]['metadata']['deci']
# 
# X = np.concatenate((X,X2), axis = 0)
# y = np.concatenate((y,y2))
# 
# print(X.shape, y.shape)
# for i in range(1, nSubj):
# 
#     X_temp = all_rawdata[i]['epoch_dat']
#     X_temp = X_temp.reshape((X_temp.shape[0]*X_temp.shape[1], X_temp.shape[2]))
#     shapes[i+1] = shapes[i] + X_temp.shape[0]
#     y_temp = all_rawdata[i]['metadata']['deci']
#     y_temp = np.tile(y_temp, numC)
# 
#     X = np.vstack((X, X_temp))
#     y = np.concatenate((y, y_temp))
# 
# print(shapes)
# 
# print(X.shape, y.shape)
# if add_labels:
#     labels = np.zeros((nSubj, X.shape[0]))
#     for i in range (nSubj):
#         for j in range(shapes[i], shapes[i+1]):
#             labels[i,j] = 1
#     print(labels.shape)
#     
#     labels = np.transpose(labels)
#     X = np.hstack((X, labels))
#     print(X.shape, labels.shape)